<a href="https://colab.research.google.com/github/nyanta012/open-model-tutorial/blob/main/section3/Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colabの使い方

- **セルの実行:** `Ctrl + Enter`, `Shift + Enter`
- **セルの作成:** `Ctrl + M B`
- **セルの削除:** `Ctrl + M D`

※GPUの設定が必要

# モデルのダウンロード

In [ ]:
%%capture
!pip install transformers==4.34 accelerate==0.25.0

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-instruct
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# プロンプトの設定

In [ ]:
def generate_prompt(question: str) -> str:
  # ELYZAのモデルはLlama2を日本語化したものなので、Llama2のInstruction Tuningの形式に準拠している
  prompt = f"""<s>[INST] <<SYS>>
  あなたは誠実で優秀な日本人のアシスタントです。
  <</SYS>>

  {question} [/INST]"""
  return prompt

# 文章の生成

In [ ]:
def inference(prompt: str) -> None:
  with torch.no_grad():
      token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

      output_ids = model.generate(
          input_ids=token_ids.to(model.device),
          max_new_tokens=256,
          pad_token_id=tokenizer.pad_token_id,
          eos_token_id=tokenizer.eos_token_id,
      )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
  print(output)

In [ ]:
prompt = generate_prompt("東京都のにゃんた高校の校歌を作成してください")
inference(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


 東京都のにゃんた高校の校歌を作成します。

校歌の歌詞は以下の通りです。

猫の姿で学ぶ猫たち
知識と忍耐を身につけて
世界に羽ばたく力を
このにゃんた高校で身につけよう

校歌のメロディーは以下の通りです。

♪にゃーにゃーにゃーにゃーにゃーにゃー
♪にゃーにゃーにゃーにゃーにゃーにゃー
♪にゃーにゃーにゃーにゃーにゃーにゃー
♪にゃーにゃーにゃーにゃーにゃーにゃー

これらは一例ですので、ご注意下さい。


# 翻訳

In [ ]:
prompt = generate_prompt("以下の日本語を英訳してください: ```私はUdemyの動画を見て大規模言語モデルをGoogleColabで動かす方法を勉強をしています```")
inference(prompt)

 I am studying how to run a large language model on Google Colab after watching a Udemy video.


In [ ]:
prompt = generate_prompt("以下の日本語を英訳してください: ```NVIDIAのGPUは非常に高価らしいですね```")
inference(prompt)

 NVIDIAのGPUは非常に高価であるようですね


In [ ]:
prompt = generate_prompt("以下の日本語を英訳してください: ```ChatGPTでなくてオープンソースのモデルは、モデルによってうまく動くものとそうでないものもあるようです```")
inference(prompt)

 The chatGPT model, unlike open-source models, can sometimes work well and sometimes not work well.
